In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
%memit tree = Tree(fc.eightb.preselection.Bkg_MC_List)

peak memory: 302.05 MiB, increment: 127.55 MiB


In [4]:
def get_component_fields(tree, pattern):
    fields = list(filter(lambda field: re.match(pattern, field), tree.fields))
    components = dict.fromkeys([re.search(pattern, field)[0]
                                for field in fields if re.search(pattern, field)]).keys()

    shared_fields = list(set.intersection(*[set(map(lambda field: field[len(component)+1:], filter(
        lambda field: field.startswith(component), fields))) for component in components]))
    components = [get_collection(tree, component, named=False)[
        shared_fields] for component in components]
    
    return components,shared_fields

In [5]:
name = 'bjet'
pattern = 'H\dY\d_b\d'


In [6]:
tree.ttree.caches

(<LRUArrayCache (0/100000000 bytes full) at 0x7f8b27b04400>,)

In [7]:
%memit get_component_fields(tree, pattern)

peak memory: 301.34 MiB, increment: 0.20 MiB


In [8]:
%memit components, shared_fields = get_component_fields(tree, pattern)

peak memory: 302.23 MiB, increment: 0.81 MiB


In [31]:
field = 'phi'

In [53]:
%memit arrays = [ component[field][:,None] for component in components ]

peak memory: 1071.28 MiB, increment: 0.00 MiB


In [54]:
array = ak.virtual(ak.concatenate, args=(arrays,), kwargs={'axis':-1})

In [78]:
%memit array = ak.concatenate(arrays,axis=-1)

peak memory: 1264.91 MiB, increment: 0.00 MiB


In [91]:
class virtual_collection:
    def __init__(self, collection):
        self.collection = collection
    def __call__(self):
        return ak.Array(self.collection)

In [93]:
array = ak.virtual( virtual_collection(components[0].pt) )

In [94]:
array

ValueError: content argument must be a Content subtype

(https://github.com/scikit-hep/awkward-1.0/blob/1.7.0/src/python/content.cpp#L342)

In [83]:
def genarray():
    return ak.Array([1,2,3,4])

In [84]:
array = ak.virtual(genarray)

In [87]:
array = components[0].pt

In [90]:
ak.Array(array)

<Array [105, 120, 57.4, 100, ... 101, 291, 68] type='2638036 * float32'>